## Web Search Ant As Tool

This example demonstrates one ant using another ant as a tool so that the second ant's work is outside the main thread.

The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [12]:
#r "nuget: AntRunnerLib, 0.8.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.6

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [13]:
var assistantId = await AssistantUtility.Create("WebSearchAgentAnt", config);
Console.WriteLine(assistantId)

asst_qZwWwscFf9ypKr3B78SfIRnq


In [14]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [15]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')
today_date
```

I called the tool named crawl with these arguments:
```
{
  "q": "things to do in Atlanta on September 24, 2024",
  "answerCount": 5,
  "textFormat": "HTML"
}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on September 24, 2024"},"webPages":{"value":[{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... Sep 24, 2024 6:00 PM . Edwin Raphael . Upcoming Events: 2. Altar at Masquerade - GA . Sep 24, 2024 7:00 PM ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"Things to do in Atlanta, GA in September, including upcoming festivals, Labor Day Weekend events, Pride events, neighborhood events, comedy, musicals, races, conventions, shows and more. ... \u2013 Geeked Week 24, September 19 2024 Netflix preview event at The Eastern. More details \u2013 Georgia Latino International Film Festival, September 19-22 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=-nsAOlbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Stay up to date on things to do and events in Atlanta today, this week and this weekend. Whether you\u2019re into live music, food and wine, sports or family-friendly entertainment, Atlanta events offer nonstop fun for everyone. ... Venture Atlanta 2024 10/8/24 \u2013 10/9/24. Atlanta miniFEST 10/9/24 \u2013 10/10/24. Homeshake ... September 24. Tuesday ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"24 Atlanta festivals to enjoy in September","url":"https://www.atlantamagazine.com/news-culture-articles/24-atlanta-festivals-to-enjoy-in-september/","displayUrl":"https://www.atlantamagazine.com/.../24-atlanta-festivals-to-enjoy-in-september","snippet":"Anime Weekend Atlanta 9/24\u201327 Autumn\u2019s answer to MomoCon, attend anime and manga panels, concerts, screenings, and more in Cobb County. ... September 18, 2024. Finding salvation at the bottom ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4664725784965333\u0026mkt=en-US\u0026setlang=en-US\u0026w=72Fb-bIAeDJJRea4C4ZweP1OtoDnBImF"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Jazz Festival \u2013 May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15\u002B artists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food \u0026amp; Wine Festival highlights the best food, wine, beer \u0026amp; cocktails Atlanta has to offer. This unique event brings in 150\u002B chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines \u0026amp; special Tasting Tents.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4642636760352967\u0026mkt=en-US\u0026setlang=en-US\u0026w=CrMoojBamU4_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find things to do in Atlanta on September 24, 2024. https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"The information about things to do in Atlanta on September 24, 2024, was not found on the page. However, here are some general events and activities happening in Atlanta in September:\n\n1. **Atlanta Food \u0026 Wine Festival** (Sept. 12-15)\n2. **Burger Benefit** (Sept. 22)\n3. **NETHERWORLD Haunted House** (Opens weekends starting Sept. 20-21)\n4. **Harvest on the Hooch** (Sept. 22)\n5. **Fiestas Patrias Celebration** (Sept. 17)\n6. **Sweet Auburn Music Fest** (Sept. 28-29)\n7. **Buckhead Fine Arts Festival** (Sept. 21-22)\n8. **JapanFest** (Sept. 21-22)\n\nFor more detailed information, you can visit the [Discover Atlanta page](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find things to do in Atlanta on September 24, 2024. https://atlanta-ga.events/september/"}
```
and got this result:
```
"Here are some events happening in Atlanta on September 24, 2024:\n\n1. **Signs of The Swarm**\n   - **Time:** 6:00 PM\n   - **Venue:** Hell at The Masquerade - GA\n   - **Address:** 75 MLK Jr Dr SW, Atlanta, GA, US\n   - **Prices:** From $37\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/signs-of-the-swarm/)\n\n2. **Kehlani**\n   - **Time:** 7:00 PM\n   - **Venue:** Lakewood Amphitheatre\n   - **Address:** 2002 Lakewood Avenue Southeast, Atlanta, GA, US\n   - **Prices:** From $51\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/kehlani/)\n\n3. **Atlanta Braves vs. New York Mets**\n   - **Time:** 7:20 PM\n   - **Venue:** Truist Park\n   - **Address:** 1100 Circle 75 Pkwy, Atlanta, GA, US\n   - **Prices:** From $15\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/new-york-mets/)\n\n4. **Atlanta United II vs. Inter Miami II**\n   - **Time:** 7:30 PM\n   - **Venue:** Fifth Third Stadium\n   - **Address:** 3200 Busbee Pkwy, Kennesaw, GA, US\n   - **Prices:** From $21\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/atlanta-united-ii/)\n\n5. **Nervosa**\n   - **Time:** 7:30 PM\n   - **Venue:** Boggs Social \u0026 Supply\n   - **Address:** 1310 White St. SW, Atlanta, GA, US\n   - **Prices:** From $73\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/nervosa/)\n\n6. **DPR**\n   - **Time:** 8:00 PM\n   - **Venue:** Coca-Cola Roxy\n   - **Address:** 800 Battery Ave SE, Atlanta, GA, US\n   - **Prices:** From $53\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/dpr-live/)\n\n7. **Victoria Canal**\n   - **Time:** 8:00 PM\n   - **Venue:** Vinyl - GA\n   - **Address:** 1374 West Peachtree Street, Atlanta, GA, US\n   - **Prices:** From $31\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/victoria-canal/)\n\n8. **Juan Fernando Velasco**\n   - **Time:** 8:00 PM\n   - **Venue:** City Winery - Atlanta\n   - **Address:** 675 Ponce De Leon Ave, Atlanta, GA, US\n   - **Prices:** From $89\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/juan-fernando-velasco/)\n\n9. **Sofi Tukker**\n   - **Time:** 8:00 PM\n   - **Venue:** The Tabernacle - GA\n   - **Address:** 152 Luckie St, Atlanta, GA, US\n   - **Prices:** From $31\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/sofi-tukker/)\n\n10. **Magdalena Bay**\n    - **Time:** 8:00 PM\n    - **Venue:** Variety Playhouse\n    - **Address:** 1099 Euclid Ave, Atlanta, GA, US\n    - **Prices:** From $57\n    - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/magdalena-bay/)\n\n11. **Rocco**\n    - **Time:** 8:00 PM\n    - **Venue:** Aisle 5\n    - **Address:** 1123 Euclid Ave Ne, Atlanta, GA, US\n    - **Prices:** From $26\n    - **Tickets:** [Get Tickets](https://atlanta-ga.events/tickets/rocco/)\n\nFor more events, visit the [Atlanta Events September 2024](https://atlanta-ga.events/september/) page."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find things to do in Atlanta on September 24, 2024. https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html"}
```
and got this result:
```
"Here are some events happening in Atlanta on September 24, 2024:\n\n1. **Chaucer\u2019s The Canterbury Tales**\n   - Play at the Shakespeare Tavern Playhouse. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n2. **Daddy Long Legs**\n   - Musical at The Legacy Theatre, Tyrone. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n3. **Boh\u00E8me Project: La boh\u00E8me**\n   - Opera at Pullman Yards. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n4. **The Chinese Lady**\n   - Play at Alliance Theatre. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n5. **North Georgia State Fair**\n   - Annual fair at Jim R Miller Park, Marietta, with carnival rides, animal shows, performances, rodeo, car events, and more. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n6. **Boh\u00E8me Project: Rent**\n   - Opera at Pullman Yards. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n7. **Little Shop of Horrors: An Art Farm at Serenbe and Matt Logan Production**\n   - Musical under the tent in the Wildflower Meadow at Serenbe, Chattahoochee Hills. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\n8. **The Curious Cardinal**\n   - Children\u2019s performance at Alliance Theatre. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)\n\nFor more details on these events, please visit the [Atlanta in September Events page](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find things to do in Atlanta on September 24, 2024. https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/"}
```
and got this result:
```
"Here are some things to do in Atlanta on September 24, 2024:\n\n### Attend a Concert\n- **SOFI TUKKER**: Performing at Tabernacle on September 24, 2024.\n\n### Other Events in September 2024\n- **Cheer on the Atlanta Braves**: Attend a home game at Truist Park.\n- **Taste of Nigeria**: Experience culture and cuisine at A Taste of Nigeria on September 28, 2024.\n- **Experience Fernbank...but Later**: Explore exhibits with later access to Fernbank on September 27, 2024.\n- **Taste of Soul Festival**: Enjoy Soul food and culture on August 30 and September 1, 2024.\n- **Grant Park Food Walk**: Try eight modern and Southern foods on a walking tour showcasing Atlanta\u0027s food and history.\n\nFor more details, you can visit the [Unexpected Atlanta page](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "Find things to do in Atlanta on September 24, 2024. https://secretatlanta.co/things-to-do-september/"}
```
and got this result:
```
"It seems that I am unable to access the content of the provided URL due to a restriction (403 Forbidden error). You might want to check the URL or provide another source for the information you need."
```

Here are some exciting things to do in Atlanta on September 24, 2024:

### Concerts and Shows
1. **Signs of The Swarm**
   - **Time:** 6:00 PM
   - **Venue:** Hell at The Masquerade - GA
   - **Address:** 75 MLK Jr Dr SW, Atlanta, GA, US
   - **Prices:** From $37
   - **[Get Tickets](https://atlanta-ga.events/tickets/signs-of-the-swarm/)

2. **Kehlani**
   - **Time:** 7:00 PM
   - **Venue:** Lakewood Amphitheatre
   - **Address:** 2002 Lakewood Avenue Southeast, Atlanta, GA, US
   - **Prices:** From $51
   - **[Get Tickets](https://atlanta-ga.events/tickets/kehlani/)

3. **Atlanta Braves vs. New York Mets**
   - **Time:** 7:20 PM
   - **Venue:** Truist Park
   - **Address:** 1100 Circle 75 Pkwy, Atlanta, GA, US
   - **Prices:** From $15
   - **[Get Tickets](https://atlanta-ga.events/tickets/new-york-mets/)

4. **DPR**
   - **Time:** 8:00 PM
   - **Venue:** Coca-Cola Roxy
   - **Address:** 800 Battery Ave SE, Atlanta, GA, US
   - **Prices:** From $53
   - **[Get Tickets](https://atlanta-ga.events/tickets/dpr-live/)

5. **Victoria Canal**
   - **Time:** 8:00 PM
   - **Venue:** Vinyl - GA
   - **Address:** 1374 West Peachtree Street, Atlanta, GA, US
   - **Prices:** From $31
   - **[Get Tickets](https://atlanta-ga.events/tickets/victoria-canal/)

### Theater and Performances
1. **Chaucer’s The Canterbury Tales**
   - **Venue:** Shakespeare Tavern Playhouse
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

2. **Daddy Long Legs**
   - **Venue:** The Legacy Theatre, Tyrone
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

3. **Bohème Project: La bohème**
   - **Venue:** Pullman Yards
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

4. **The Chinese Lady**
   - **Venue:** Alliance Theatre
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

### Festivals and Fairs
1. **North Georgia State Fair**
   - **Venue:** Jim R Miller Park, Marietta
   - **Activities:** Carnival rides, animal shows, performances, rodeo, car events, and more.
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

### Additional Activities
1. **SOFI TUKKER**: Performing at Tabernacle on September 24, 2024.
2. **Grant Park Food Walk**: Try eight modern and Southern foods on a walking tour showcasing Atlanta's food and history.

For more information and additional events, you can visit the [Atlanta Events September 2024](https://atlanta-ga.events/september/) page or [Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/).

Enjoy your time in Atlanta!

![Atlanta](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)


In [16]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting things to do in Atlanta on September 24, 2024:

### Concerts and Shows
1. **Signs of The Swarm**
   - **Time:** 6:00 PM
   - **Venue:** Hell at The Masquerade - GA
   - **Address:** 75 MLK Jr Dr SW, Atlanta, GA, US
   - **Prices:** From $37
   - **[Get Tickets](https://atlanta-ga.events/tickets/signs-of-the-swarm/)

2. **Kehlani**
   - **Time:** 7:00 PM
   - **Venue:** Lakewood Amphitheatre
   - **Address:** 2002 Lakewood Avenue Southeast, Atlanta, GA, US
   - **Prices:** From $51
   - **[Get Tickets](https://atlanta-ga.events/tickets/kehlani/)

3. **Atlanta Braves vs. New York Mets**
   - **Time:** 7:20 PM
   - **Venue:** Truist Park
   - **Address:** 1100 Circle 75 Pkwy, Atlanta, GA, US
   - **Prices:** From $15
   - **[Get Tickets](https://atlanta-ga.events/tickets/new-york-mets/)

4. **DPR**
   - **Time:** 8:00 PM
   - **Venue:** Coca-Cola Roxy
   - **Address:** 800 Battery Ave SE, Atlanta, GA, US
   - **Prices:** From $53
   - **[Get Tickets](https://atlanta-ga.events/tickets/dpr-live/)

5. **Victoria Canal**
   - **Time:** 8:00 PM
   - **Venue:** Vinyl - GA
   - **Address:** 1374 West Peachtree Street, Atlanta, GA, US
   - **Prices:** From $31
   - **[Get Tickets](https://atlanta-ga.events/tickets/victoria-canal/)

### Theater and Performances
1. **Chaucer’s The Canterbury Tales**
   - **Venue:** Shakespeare Tavern Playhouse
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

2. **Daddy Long Legs**
   - **Venue:** The Legacy Theatre, Tyrone
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

3. **Bohème Project: La bohème**
   - **Venue:** Pullman Yards
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

4. **The Chinese Lady**
   - **Venue:** Alliance Theatre
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

### Festivals and Fairs
1. **North Georgia State Fair**
   - **Venue:** Jim R Miller Park, Marietta
   - **Activities:** Carnival rides, animal shows, performances, rodeo, car events, and more.
   - **More details:** [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)

### Additional Activities
1. **SOFI TUKKER**: Performing at Tabernacle on September 24, 2024.
2. **Grant Park Food Walk**: Try eight modern and Southern foods on a walking tour showcasing Atlanta's food and history.

For more information and additional events, you can visit the [Atlanta Events September 2024](https://atlanta-ga.events/september/) page or [Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/).

Enjoy your time in Atlanta!

![Atlanta](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)

In [17]:
var output2 = await AntRunnerLib.AssistantRunner.RunThread("WebSearchAgentAnt", "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and links. Be careful to not include events in the past in your final answer. You should perform four searches at once.");
output2.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta over the next four weekends:

### Weekend of September 28-29, 2024:
1. **Sweet Auburn Music Fest**
   - **Date**: September 28-29
   - **Location**: Auburn Avenue
   - **Description**: The festival offers fun-filled entertainment, live music (everything from R&B to gospel to alternative favorites), international food court, national artists, celebrities, and vendors of all types.
   - ![Sweet Auburn Music Fest](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)
   - [More details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

2. **Baldie Con**
   - **Date**: September 27-29
   - **Location**: JW Marriott Atlanta Buckhead
   - **Description**: An empowerment conference for the bald community, celebrating and uniting women and children who are bald due to alopecia, lupus, cancer, or by choice. The event includes a Baldie Ball with an award ceremony and gala dinner on Saturday night and a Sunday Jazz Brunch on September 29.
   - ![Baldie Con](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)
   - [More details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

### Weekend of October 5-6, 2024:
1. **Festival on Ponce**
   - **Date**: October 5-6
   - **Location**: Olmsted Linear Park
   - **Description**: Twice-annual arts festival in Olmsted Linear Park, with 125+ vendors of fine art, folk art, and outsider art, plus food and drink, and children’s activities.
   - ![Festival on Ponce](https://www.bing.com/th?id=OIP.HSldVQBYX0bVWUIrHx7P-QHaE7&w=80&h=80&c=1&pid=5.1)
   - [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

2. **Candler Park Fall Fest**
   - **Date**: October 5-6
   - **Location**: Candler Park
   - **Description**: This popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more.
   - ![Candler Park Fall Fest](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)
   - [More details](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

### Weekend of October 12-13, 2024:
1. **Oakhurst Porchfest**
   - **Date**: October 12
   - **Location**: Decatur
   - **Description**: Annual community music festival in Decatur with over 100 live performances on Oakhurst neighborhood porches.
   - ![Oakhurst Porchfest](https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1)
   - [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

2. **Atlanta Pride Festival**
   - **Date**: October 12-13
   - **Location**: Piedmont Park
   - **Description**: One of the biggest Pride festivals in the country, featuring live performances, parade, marches, marketplace, and more.
   - ![Atlanta Pride Festival](https://www.bing.com/th?id=OIP.Jx474uFzP4iYwa3hHq7X7AHaFn&w=80&h=80&c=1&pid=5.1)
   - [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

### Weekend of October 19-20, 2024:
1. **L5P Halloween Festival and Parade**
   - **Date**: October 19-20
   - **Location**: Little Five Points
   - **Description**: Popular annual Halloween celebration in Little Five Points, with live music, parade, haunted house, ghost tours, and more.
   - ![L5P Halloween Festival](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)
   - [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

2. **PNC Atlanta 10 Miler & 5K**
   - **Date**: October 19-20
   - **Location**: Buckhead, starting from Atlantic Station
   - **Description**: Annual 10-mile, 5K, and 1-mile youth runs.
   - ![PNC Atlanta 10 Miler & 5K](https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1)
   - [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

Enjoy your time in Atlanta with these wonderful events!

## Clean Up

In [18]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebSearchAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
